#### Model loading and prediction script

In [11]:
import pandas as pd
import xgboost as xgb
import json
from typing import Dict, Any

class FraudPredictor:
    def __init__(self, model_path: str):
        """Initialize the predictor with the trained model."""
        self.model = xgb.XGBClassifier()
        self.model.load_model(model_path)  # Load XGBoost model
        self.required_features = self.model.feature_names_in_.tolist()  # Get feature names
        self.threshold = 0.3  # Adjust based on your precision/recall needs

    def validate_input(self, transaction_data: Dict[str, Any]) -> pd.DataFrame:
        """Validate and convert input data to DataFrame with correct feature order."""
        # Check for missing features
        missing_features = set(self.required_features) - set(transaction_data.keys())
        if missing_features:
            raise ValueError(f"Missing features: {missing_features}")

        # Create DataFrame with exact feature order used in training
        return pd.DataFrame([transaction_data])[self.required_features]

    def predict_fraud(self, transaction_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Predict fraud probability for a transaction.
        
        Args:
            transaction_data: Dictionary containing all features used in training.
                             Example:
                             {
                                 "Amount": 250.00,
                                 "Type of Transaction": 0.3174,
                                 "Merchant Group": 0.177383592,
                                 "is_foreign": 1,
                                 "is_night": 1,
                                 "bank_risk_score": 0.18128655,
                                 "Type of Card_MasterCard": "TRUE",
                                 "Type of Card_Visa": "FALSE",
                                 "Entry Mode_CVC": "FALSE",
                                 "Entry Mode_PIN": "TRUE",
                                 "Entry Mode_Tap": "FALSE"
                             }
        
        Returns:
            Dictionary with prediction results.
        """
        try:
            # Convert string booleans to integers
            converted_data = {}
            for key, value in transaction_data.items():
                if isinstance(value, str):
                    if value.upper() == "TRUE":
                        converted_data[key] = 1
                    elif value.upper() == "FALSE":
                        converted_data[key] = 0
                    else:
                        converted_data[key] = float(value)  # For other string numbers
                else:
                    converted_data[key] = value
            
            # Validate and prepare input
            input_df = self.validate_input(converted_data)
            
            # Get probability
            prob = self.model.predict_proba(input_df)[0][1]  # P(Fraud = 1)
            
            return {
                "fraud_probability": float(prob),
                "is_fraud": bool(prob > self.threshold),
                "threshold": self.threshold,
                "status": "success"
            }
        except Exception as e:
            return {
                "status": "error",
                "message": str(e)
            }

# Initialize predictor
MODEL_PATH = r"C:\Seshere\ML\Imperial\25 Module - Capstone Portfolio Project\Full_Data\fraud_model.xgb"
predictor = FraudPredictor(MODEL_PATH)

# Example usage
if __name__ == "__main__":
    # Sample transaction 
    transaction = {
        "Amount": 5680.00,
        "Type of Transaction": 0.3242,
        "Merchant Group": 0.177384,
        "is_foreign": 1,
        "is_night": 1,
        "bank_risk_score": 0.181,
        "Type of Card_MasterCard": "FALSE",
        "Type of Card_Visa": "TRUE",
        "Entry Mode_CVC": "FALSE",
        "Entry Mode_PIN": "TRUE",
        "Entry Mode_Tap": "FALSE"
    }

    # Make prediction
    result = predictor.predict_fraud(transaction)
    print(json.dumps(result, indent=2))

{
  "fraud_probability": 0.9857153296470642,
  "is_fraud": true,
  "threshold": 0.3,
  "status": "success"
}
